## To Do

* [x] vector
* [x] list
* [ ] iterator
* [ ] forward_list
* [ ] queue
* [ ] deque
* [ ] map
* [ ] set
* [ ] unordered_map
* [ ] multi_map
* [ ] multi_set
* [ ] hash
* [ ] function

### Basic

- https://stackoverflow.com/questions/14148756/what-does-template-rebind-do

- https://www.bbsmax.com/A/lk5aNgq2d1/

  ![rebind](./Images/rebind.png)

  ```cpp
  	std::allocator<uint32_t>::rebind<char8_t>::other allocatorTest;
  ```

  

```cpp
template<class _Ty>
	struct remove_reference
	{	// remove reference
	using type = _Ty;
	};

template<class _Ty>
	struct remove_reference<_Ty&>
	{	// remove reference
	using type = _Ty;
	};

template<class _Ty>
	struct remove_reference<_Ty&&>
	{	// remove rvalue reference
	using type = _Ty;
	};

template<class _Ty>
	using remove_reference_t = typename remove_reference<_Ty>::type;

template<class _Ty>
	struct remove_cv
	{	// remove top level const and volatile qualifiers
	using type = _Ty;
	};

template<class _Ty>
	struct remove_cv<const _Ty>
	{	// remove top level const and volatile qualifiers
	using type = _Ty;
	};

template<class _Ty>
	struct remove_cv<volatile _Ty>
	{	// remove top level const and volatile qualifiers
	using type = _Ty;
	};

template<class _Ty>
	struct remove_cv<const volatile _Ty>
	{	// remove top level const and volatile qualifiers
	using type = _Ty;
	};

template<class _Ty>
	using remove_cv_t = typename remove_cv<_Ty>::type;

template<class _Ty>
	struct remove_extent
	{	// remove array extent
	using type = _Ty;
	};

template<class _Ty, size_t _Ix>
	struct remove_extent<_Ty[_Ix]>
	{	// remove array extent
	using type = _Ty;
	};

template<class _Ty>
	struct remove_extent<_Ty[]>
	{	// remove array extent
	using type = _Ty;
	};

template<class _Ty>
	using remove_extent_t = typename remove_extent<_Ty>::type;
```

```cpp
template<class _Ty>
	_NODISCARD constexpr remove_reference_t<_Ty>&&
		move(_Ty&& _Arg) noexcept
	{	// forward _Arg as movable
	return (static_cast<remove_reference_t<_Ty>&&>(_Arg));
	}
```

```cpp
template<class _Ty>
	_NODISCARD constexpr _Ty&& forward(remove_reference_t<_Ty>& _Arg) noexcept
	{	// forward an lvalue as either an lvalue or an rvalue
	return (static_cast<_Ty&&>(_Arg));
	}

template<class _Ty>
	_NODISCARD constexpr _Ty&& forward(remove_reference_t<_Ty>&& _Arg) noexcept
	{	// forward an rvalue as an rvalue
	static_assert(!is_lvalue_reference_v<_Ty>, "bad forward call");
	return (static_cast<_Ty&&>(_Arg));
	}
```

### std::vector

```cpp
template<class _InIt,
	class _OutIt> inline
	_OutIt _Move_unchecked1(_InIt _First, _InIt _Last,
		_OutIt _Dest, _General_ptr_iterator_tag)
	{	// move [_First, _Last) to [_Dest, ...), no special optimization
	for (; _First != _Last; ++_Dest, (void)++_First)
		*_Dest = _STD move(*_First);
	return (_Dest);
	}

template<class _InIt,
	class _OutIt> inline
	_OutIt _Move_unchecked1(_InIt _First, _InIt _Last,
		_OutIt _Dest, _Trivially_copyable_ptr_iterator_tag)
	{	// move [_First, _Last) to [_Dest, ...), memmove optimization
	return (_Copy_memmove(_First, _Last, _Dest));
	}
```



```cpp
	_NODISCARD size_type capacity() const noexcept
		{	// return current length of allocated storage
		return (static_cast<size_type>(this->_Myend() - this->_Myfirst()));
		}
```



```cpp
	size_type _Calculate_growth(const size_type _Newsize) const
		{	// given _Oldcapacity and _Newsize, calculate geometric growth
		const size_type _Oldcapacity = capacity();

		if (_Oldcapacity > max_size() - _Oldcapacity / 2)
			{
			return (_Newsize);	// geometric growth would overflow
			}

		const size_type _Geometric = _Oldcapacity + _Oldcapacity / 2;

		if (_Geometric < _Newsize)
			{
			return (_Newsize);	// geometric growth would be insufficient
			}

		return (_Geometric);	// geometric growth is sufficient
		}
```

```cpp
	_NODISCARD _Ty& front()
		{	// return first element of mutable sequence
 #if _ITERATOR_DEBUG_LEVEL != 0
		_STL_VERIFY(!empty(), "front() called on empty vector");
 #endif /* _ITERATOR_DEBUG_LEVEL != 0 */

		return (*this->_Myfirst());
		}

	_NODISCARD const _Ty& front() const
		{	// return first element of nonmutable sequence
 #if _ITERATOR_DEBUG_LEVEL != 0
		_STL_VERIFY(!empty(), "front() called on empty vector");
 #endif /* _ITERATOR_DEBUG_LEVEL != 0 */

		return (*this->_Myfirst());
		}

	_NODISCARD _Ty& back()
		{	// return last element of mutable sequence
 #if _ITERATOR_DEBUG_LEVEL != 0
		_STL_VERIFY(!empty(), "back() called on empty vector");
 #endif /* _ITERATOR_DEBUG_LEVEL != 0 */

		return (this->_Mylast()[-1]);
		}

	_NODISCARD const _Ty& back() const
		{	// return last element of nonmutable sequence
 #if _ITERATOR_DEBUG_LEVEL != 0
		_STL_VERIFY(!empty(), "back() called on empty vector");
 #endif /* _ITERATOR_DEBUG_LEVEL != 0 */

		return (this->_Mylast()[-1]);
		}

	void clear() noexcept
		{	// erase all
		this->_Orphan_all();
		_Destroy(this->_Myfirst(), this->_Mylast());
		this->_Mylast() = this->_Myfirst();
		}

	iterator erase(const_iterator _Where)
		{	// erase element at _Where
 #if _ITERATOR_DEBUG_LEVEL == 2
		_STL_VERIFY(_Where._Getcont() == _STD addressof(this->_Get_data())
			&& _Where._Ptr >= this->_Myfirst()
			&& this->_Mylast() > _Where._Ptr, "vector erase iterator outside range");
		_Orphan_range(_Where._Ptr, this->_Mylast());
 #endif /* _ITERATOR_DEBUG_LEVEL == 2 */

		_Move_unchecked(_Where._Ptr + 1, this->_Mylast(), _Where._Ptr);
		_Alty_traits::destroy(this->_Getal(), _Unfancy(this->_Mylast() - 1));
		--this->_Mylast();
		return (iterator(_Where._Ptr, _STD addressof(this->_Get_data())));
		}

	iterator erase(const_iterator _First, const_iterator _Last)
		{	// erase [_First, _Last)
 #if _ITERATOR_DEBUG_LEVEL == 2
		_STL_VERIFY(_First._Getcont() == _STD addressof(this->_Get_data())
			&& _Last._Getcont() == _STD addressof(this->_Get_data())
			&& _First._Ptr >= this->_Myfirst()
			&& _Last._Ptr >= _First._Ptr
			&& this->_Mylast() >= _Last._Ptr, "vector erase iterator outside range");
 #endif /* _ITERATOR_DEBUG_LEVEL == 2 */

		if (_First._Ptr != _Last._Ptr)
			{	// something to do, invalidate iterators
			_Orphan_range(_First._Ptr, this->_Mylast());
			const pointer _Newlast = _Move_unchecked(_Last._Ptr, this->_Mylast(), _First._Ptr);
			_Destroy(_Newlast, this->_Mylast());
			this->_Mylast() = _Newlast;
			}

		return (iterator(_First._Ptr, _STD addressof(this->_Get_data())));
		}
```

```cpp
	void push_back(const _Ty& _Val)
		{	// insert element at end, provide strong guarantee
		emplace_back(_Val);
		}

	void push_back(_Ty&& _Val)
		{	// insert by moving into element at end, provide strong guarantee
		emplace_back(_STD move(_Val));
		}

	template<class... _Valty>
		decltype(auto) _Emplace_back_with_unused_capacity(_Valty&&... _Val)
		{	// insert by perfectly forwarding into element at end, provide strong guarantee
			// pre: _Has_unused_capacity()
		_Alty_traits::construct(this->_Getal(), _Unfancy(this->_Mylast()), _STD forward<_Valty>(_Val)...);
		_Orphan_range(this->_Mylast(), this->_Mylast());
		_Ty& _Result = *this->_Mylast();
		++this->_Mylast();
#if _HAS_CXX17
		return (_Result);
#else /* ^^^ _HAS_CXX17 ^^^ // vvv !_HAS_CXX17 vvv */
		(void)_Result;
#endif /* _HAS_CXX17 */
		}

	template<class... _Valty>
		decltype(auto) emplace_back(_Valty&&... _Val)
		{	// insert by perfectly forwarding into element at end, provide strong guarantee
		if (_Has_unused_capacity())
			{
			return (_Emplace_back_with_unused_capacity(_STD forward<_Valty>(_Val)...));
			}

		_Ty& _Result = *_Emplace_reallocate(this->_Mylast(), _STD forward<_Valty>(_Val)...);
#if _HAS_CXX17
		return (_Result);
#else /* ^^^ _HAS_CXX17 ^^^ // vvv !_HAS_CXX17 vvv */
		(void)_Result;
#endif /* _HAS_CXX17 */
		}
```

### std::list

```cpp
template<class _Value_type,
	class _Voidptr>
	struct _List_node
		{	// list node
		using _Nodeptr = _Rebind_pointer_t<_Voidptr, _List_node>;
		_Nodeptr _Next;	// successor node, or first element if head
		_Nodeptr _Prev;	// predecessor node, or last element if head
		_Value_type _Myval;	// the stored value, unused if head

		_List_node& operator=(const _List_node&) = delete;
		};
```

```cpp
	void push_front(_Ty&& _Val)
		{	// insert element at beginning
		_Insert(_Unchecked_begin(), _STD move(_Val));
		}

	void push_back(_Ty&& _Val)
		{	// insert element at end
		_Insert(_Unchecked_end(), _STD move(_Val));
		}

	iterator insert(const_iterator _Where, _Ty&& _Val)
		{	// insert _Val at _Where
		return (emplace(_Where, _STD move(_Val)));
		}

	template<class... _Valty>
		decltype(auto) emplace_front(_Valty&&... _Val)
		{	// insert element at beginning
		_Insert(_Unchecked_begin(), _STD forward<_Valty>(_Val)...);

#if _HAS_CXX17
		return (front());
#endif /* _HAS_CXX17 */
		}

	template<class... _Valty>
		decltype(auto) emplace_back(_Valty&&... _Val)
		{	// insert element at end
		_Insert(_Unchecked_end(), _STD forward<_Valty>(_Val)...);

#if _HAS_CXX17
		return (back());
#endif /* _HAS_CXX17 */
		}

	template<class... _Valty>
		iterator emplace(const_iterator _Where, _Valty&&... _Val)
		{	// insert element at _Where
 #if _ITERATOR_DEBUG_LEVEL == 2
		_STL_VERIFY(_Where._Getcont() == _STD addressof(this->_Get_data()), "list emplace iterator outside range");
 #endif /* _ITERATOR_DEBUG_LEVEL == 2 */

		_Insert(_Where._Unwrapped(), _STD forward<_Valty>(_Val)...);
		return (_Make_iter(--_Where));
		}

	void push_front(const _Ty& _Val)
		{	// insert element at beginning
		_Insert(_Unchecked_begin(), _Val);
		}

	void pop_front()
		{	// erase element at beginning
		erase(begin());
		}

	void push_back(const _Ty& _Val)
		{	// insert element at end
		_Insert(_Unchecked_end(), _Val);
		}

	void pop_back()
		{	// erase element at end
		erase(--end());
		}
```

```cpp
	template<class... _Valty>
		void _Insert(_Unchecked_const_iterator _Where, _Valty&&... _Val)
		{	// insert element at _Where
		const _Nodeptr _Rightnode = _Where._Ptr;
		const _Nodeptr _Leftnode = _Rightnode->_Prev;
		const _Nodeptr _Newnode = this->_Buynode(_Rightnode, _Leftnode, _STD forward<_Valty>(_Val)...);
		_Incsize(1);
		_Rightnode->_Prev = _Newnode;
		_Leftnode->_Next = _Newnode;
		}
```

```cpp
	void remove(const _Ty& _Val)
		{	// erase each element matching _Val
		iterator _Val_it = end();

		for (iterator _First = begin(); _First != end(); )
			if (*_First == _Val)
				if (_STD addressof(*_First) == _STD addressof(_Val))
					_Val_it = _First++;
				else
					_First = erase(_First);
			else
				++_First;

		if (_Val_it != end())
			erase(_Val_it);
		}

	template<class _Pr1>
		void remove_if(_Pr1 _Pred)
		{	// erase each element satisfying _Pred
		_Remove_if(_Pred);
		}

	template<class _Pr1>
		void _Remove_if(_Pr1& _Pred)
		{	// erase each element satisfying _Pred
		for (iterator _First = begin(); _First != end(); )
			if (_Pred(*_First))
				_First = erase(_First);
			else
				++_First;
		}
```

```cpp
	void sort()
		{	// order sequence, using operator<
		sort(less<>());
		}

	template<class _Pr2>
		void sort(_Pr2 _Pred)
		{	// order sequence, using _Pred
		_Sort(begin(), end(), _Pred, this->_Mysize());
		}

	template<class _Pr2>
		iterator _Sort(iterator _First, iterator _Last, _Pr2& _Pred,
			size_type _Size)
		{	// order [_First, _Last), using _Pred, return new first
			// _Size must be distance from _First to _Last
		if (_Size < 2)
			return (_First);	// nothing to do

		iterator _Mid = _STD next(_First, static_cast<difference_type>(_Size >> 1));
		_First = _Sort(_First, _Mid, _Pred, _Size >> 1);
		_Mid = _Sort(_Mid, _Last, _Pred, _Size - (_Size >> 1));
		iterator _Newfirst = _First;

		for (bool _Initial_loop = true; ; _Initial_loop = false)
			{	// [_First, _Mid) and [_Mid, _Last) are sorted and non-empty
			if (_DEBUG_LT_PRED(_Pred, *_Mid, *_First))
				{	// consume _Mid
				if (_Initial_loop)
					_Newfirst = _Mid;	// update return value
				splice(_First, *this, _Mid++);
				if (_Mid == _Last)
					return (_Newfirst);	// exhausted [_Mid, _Last); done
				}
			else
				{	// consume _First
				++_First;
				if (_First == _Mid)
					return (_Newfirst);	// exhausted [_First, _Mid); done
				}
			}
		}
```

 ### std::hash

- http://www.isthe.com/chongo/tech/comp/fnv/index.html#FNV-param

```cpp
#if defined(_WIN64)
_INLINE_VAR constexpr size_t _FNV_offset_basis = 14695981039346656037ULL;
_INLINE_VAR constexpr size_t _FNV_prime = 1099511628211ULL;
#else /* defined(_WIN64) */
_INLINE_VAR constexpr size_t _FNV_offset_basis = 2166136261U;
_INLINE_VAR constexpr size_t _FNV_prime = 16777619U;
#endif /* defined(_WIN64) */
```

```cpp
_NODISCARD inline size_t _Fnv1a_append_bytes(size_t _Val,
	const unsigned char * const _First, const size_t _Count) noexcept
	{	// accumulate range [_First, _First + _Count) into partial FNV-1a hash _Val
	for (size_t _Idx = 0; _Idx < _Count; ++_Idx)
		{
		_Val ^= static_cast<size_t>(_First[_Idx]);
		_Val *= _FNV_prime;
		}

	return (_Val);
	}

template<class _Kty>
	_NODISCARD inline size_t _Fnv1a_append_value(const size_t _Val, const _Kty& _Keyval) noexcept
	{	// accumulate _Keyval into partial FNV-1a hash _Val
	static_assert(is_trivial_v<_Kty>, "Only trivial types can be directly hashed.");
	return (_Fnv1a_append_bytes(_Val,
		&reinterpret_cast<const unsigned char&>(_Keyval), sizeof(_Kty)));
	}

template<class _Kty>
	_NODISCARD inline size_t _Hash_representation(const _Kty& _Keyval) noexcept
	{	// bitwise hashes the representation of a key
	return (_Fnv1a_append_value(_FNV_offset_basis, _Keyval));
	}
```

